In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center><h1><b> Exploring Historic cities </b> </h1></center>

<h2> 1. Introduction/Business Problem </h2>

<p> London and Paris are quite the popular tourist and vacation destinations for people all around the world. They are diverse and multicultural and offer a wide variety of experiences that is widely sought after. We try to group the neighbourhoods of London and Paris respectively and draw insights to what they look like now. These cities are majestic to say the least. They have been the hub of several great civilisations.</p>

<p> The aim is to help tourists choose their destinations depending on the experiences that the neighbourhoods have to offer and what they would want to have. This also helps people make decisions if they are thinking about migrating to London or Paris or even if they want to relocate neighbourhoods within the city. Our findings will help stakeholders make informed decisions and address any concerns they have including the different kinds of cuisines, provision stores and what the city has to offer. By this analysis, we provide the power of informed decision making to the viewer. We want to enable them to make informed decision and save both money and time,not to mention a lot of effort as well. </p>

<h2> 2. Data Description </h2>

<p> We require geographical location data for both London and Paris. Postal codes in each city serve as a starting point. Using Postal codes we use can find out the neighborhoods, boroughs, venues and their most popular venue categories. </p>

<p> For London Data, We scrape our data from https://en.wikipedia.org/wiki/List_of_areas_of_London

This wikipedia page has information about all the neighbourhoods, we limit it to London.
<ul>
    <li>borough : Name of Neighbourhood</li>
    <li>town : Name of borough</li>
    <li>post_code : Postal codes for London.</li>
</ul>    
</p>




<p> For Paris Data, We leverage JSON data available at https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e

The JSON file has data about all the neighbourhoods in France, we limit it to Paris.
<ul>
    <li>postal_code : Postal codes for France</li>
    <li>nom_comm : Name of Neighbourhoods in France</li>
    <li>nom_dept : Name of the boroughs, equivalent to towns in France</li>
    <li>geo_point_2d : Tuple containing the latitude and longitude of the Neighbourhoods.</li>
    </ul>
</p>


<p> We can further leverage Foursquare API to gain additional data, The data retrieved from Foursquare contained information of venues within a specified distance of the longitude and latitude of the postcodes. We set the radius to 500m. The information obtained per venue as follows:
<ul>
    <li>Neighbourhood : Name of the Neighbourhood</li>
    <li>Neighbourhood Latitude : Latitude of the Neighbourhood</li>
    <li>Neighbourhood Longitude : Longitude of the Neighbourhood</li>
    <li>Venue : Name of the Venue</li>
    <li>Venue Latitude : Latitude of Venue</li>
    <li>Venue Longitude : Longitude of Venue</li>
    <li>Venue Category : Category of Venue</li>
    </ul>
</p>


<p> This data should suffice and could very well provide us with enough resources to make a detailed report. </p>

<h2> 3. Methodology </h2>

In [73]:
#Importing libs
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# import k-means for the clustering stage
from sklearn.cluster import KMeans
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<h3>Exploring London</h3>

In [83]:
url_london = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
wiki_london_url = requests.get(url_london)
wiki_london_url

<Response [200]>

In [84]:
wiki_london_data = pd.read_html(wiki_london_url.text)
wiki_london_data

[                                                                        0
 0  Map all coordinates in "Category:Areas of London" using: OpenStreetMap
 1                                            Download coordinates as: KML,
                                            Location  \
 0                                        Abbey Wood   
 1                                             Acton   
 2                                         Addington   
 3                                        Addiscombe   
 4                                       Albany Park   
 5                                  Aldborough Hatch   
 6                                           Aldgate   
 7                                           Aldwych   
 8                                          Alperton   
 9                                           Anerley   
 10                                            Angel   
 11                                        Aperfield   
 12                                          A

In [85]:
wiki_london_data = wiki_london_data[1]
wiki_london_data.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


Preprocessing

In [86]:
df1 = wiki_london_data.drop( [ wiki_london_data.columns[0], wiki_london_data.columns[4], wiki_london_data.columns[5] ], axis=1)
df1.head()

,London borough,Post town,Postcode district
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


In [87]:
df1.columns = ['borough','town','post_code']
df1.head()

,borough,town,post_code
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


In [88]:
df1['borough'] = df1['borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df1.head()

,borough,town,post_code
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
2,Croydon,CROYDON,CR0
3,Croydon,CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


Since we're only focusing on London, removing other entries.

In [89]:
df1 = df1[df1['town'].str.contains('LONDON')]
df1.head()

,borough,town,post_code
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
6,City,LONDON,EC3
7,Westminster,LONDON,WC2
9,Bromley,LONDON,SE20


In [92]:
#df1.reset_index(inplace=True)
df1.drop('level_0',axis=1,inplace=True)
df1

,borough,town,post_code
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
2,City,LONDON,EC3
3,Westminster,LONDON,WC2
4,Bromley,LONDON,SE20
5,Islington,LONDON,"EC1, N1"
6,Islington,LONDON,N19
7,Barnet,"BARNET, LONDON","EN5, NW7"
8,Enfield,LONDON,"N11, N14"
9,Wandsworth,LONDON,SW12


In [93]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   borough    308 non-null    object
 1   town       308 non-null    object
 2   post_code  308 non-null    object
dtypes: object(3)
memory usage: 7.3+ KB


**Using Geolocation to get coordinates...**

In [51]:
!pip install geocoder

     |████████████████████████████████| 98 kB 1.0 MB/s eta 0:00:011


In [52]:
import geocoder

Function to get Coordinates via the postal code

In [54]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{},London'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords

In [77]:
get_latilong('SW14')

[51.46482000000003, -0.26590999999996257]

In [57]:
postal_codes = df1['post_code']    
coords = [ get_latilong(post_code) for post_code in postal_codes.tolist() ]

In [94]:
df_coords

,Latitude,Longitude
0,51.49245,0.121270
1,51.51324,-0.267460
2,51.51200,-0.080580
3,51.51651,-0.119680
4,51.41009,-0.056830
5,51.52361,-0.098770
6,51.56393,-0.129450
7,51.61568,-0.245110
8,51.63429,-0.133660
9,51.44822,-0.148390


In [95]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df1['Latitude'] = df_coords['Latitude']
df1['Longitude'] = df_coords['Longitude']
df1.head()

,borough,town,post_code,Latitude,Longitude
0,"Bexley, Greenwich",LONDON,SE2,51.49245,0.12127
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.51324,-0.26746
2,City,LONDON,EC3,51.51200,-0.08058
3,Westminster,LONDON,WC2,51.51651,-0.11968
4,Bromley,LONDON,SE20,51.41009,-0.05683


In [75]:
df1.shape

(308, 5)

Visualising London

In [68]:
london_lng_coords = '-0.1272099999999341'
london_lat_coords = '51.50642000000005'

In [97]:
import folium
# Creating the map of London
map_London = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)
#map_London

# adding markers to map
for latitude, longitude, borough, town in zip(df1['Latitude'], df1['Longitude'], df1['borough'], df1['town']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_London)  
    
map_London

Now we use the Foursquare API to fetch venue data

In [98]:
CLIENT_ID = '15QZR3EIH0YKGCU1MBP14NYZSMJAWC4QDG2LYLPGOMVLRNCK' 
CLIENT_SECRET = '3NCDR1HQ2EEIPIGNRWRFI21ZSTJUB42RY5AHRWZ4SWH4SXUN'
VERSION = '20180605' # Foursquare API version

In [103]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [104]:
venues_in_London = getNearbyVenues(df1['borough'], df1['Latitude'], df1['Longitude'])

Bexley, Greenwich 
Ealing, Hammersmith and Fulham
City
Westminster
Bromley
Islington
Islington
Barnet
Enfield
Wandsworth
Southwark
City
Richmond upon Thames
Barnet
Islington
Wandsworth
Westminster
Bromley
Newham
Ealing
Westminster
Lewisham
Camden
Southwark
Tower Hamlets
Bexley
City
Lewisham
Greenwich
Tower Hamlets
Camden
Haringey
Tower Hamlets
Haringey
Barnet
Brent
Lambeth
Lewisham
Tower Hamlets
Kensington and Chelsea, Hammersmith and Fulham
Brent
Barnet
Barnet
Southwark
Tower Hamlets
Camden
Tower Hamlets
Waltham Forest
Newham
Islington
Richmond upon Thames
Lewisham
Camden
Westminster
Greenwich
Kensington and Chelsea
Barnet
Westminster
Lewisham
Waltham Forest
Hounslow, Ealing, Hammersmith and Fulham
Brent
Barnet
Lambeth, Wandsworth
Islington
Barnet
Merton
Barnet
Westminster
Barnet, Brent, Camden
Lewisham
Bexley
Haringey
Bromley
Tower Hamlets
Newham
Hackney
Islington
Southwark
Lewisham
Brent
Southwark
Ealing
Kensington and Chelsea
Wandsworth
Southwark
Barnet
Newham
Richmond upon Thames


In [105]:
venues_in_London.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,"Bexley, Greenwich",51.49245,0.12127,Lesnes Abbey,Historic Site
1,"Bexley, Greenwich",51.49245,0.12127,Sainsbury's,Supermarket
2,"Bexley, Greenwich",51.49245,0.12127,Lidl,Supermarket
3,"Bexley, Greenwich",51.49245,0.12127,Abbey Wood Railway Station (ABW),Train Station
4,"Bexley, Greenwich",51.49245,0.12127,Bean @ Work,Coffee Shop


In [106]:
venues_in_London.shape

(10313, 5)

In [114]:
len(pd.unique(venues_in_London['Venue Category']))

301

One-hot encoding venue categories

In [115]:
London_venue_cat = pd.get_dummies(venues_in_London[['Venue Category']], prefix="", prefix_sep="")
London_venue_cat.head()

,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cemetery,Champagne Bar,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus School,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Restaurant,Sake Bar,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stationery Store,Steakhouse,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Co

In [116]:
London_venue_cat['Neighbourhood'] = venues_in_London['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [London_venue_cat.columns[-1]] + list(London_venue_cat.columns[:-1])
London_venue_cat = London_venue_cat[fixed_columns]

London_venue_cat.head()

,Neighbourhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cemetery,Champagne Bar,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus School,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Restaurant,Sake Bar,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stationery Store,Steakhouse,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea 

We will group the Neighbourhoods and calculate the mean venue categories value in each Neighbourhood

In [117]:
London_grouped = London_venue_cat.groupby('Neighbourhood').mean().reset_index()
London_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cemetery,Champagne Bar,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus School,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Restaurant,Sake Bar,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stationery Store,Steakhouse,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea 

In [118]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [119]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

Getting the top venue categories in London

In [120]:
# create a new dataframe for London
neighborhoods_venues_sorted_london = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_london['Neighbourhood'] = London_grouped['Neighbourhood']

for ind in np.arange(London_grouped.shape[0]):
    neighborhoods_venues_sorted_london.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_london.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barnet,Coffee Shop,Café,Grocery Store,Pub,Italian Restaurant,Bus Stop,Supermarket,Pharmacy,Turkish Restaurant,Chinese Restaurant
1,"Barnet, Brent, Camden",Gym / Fitness Center,Hardware Store,Clothing Store,Supermarket,Opera House,Park,Paper / Office Supplies Store,Pakistani Restaurant,Outdoor Sculpture,Organic Grocery
2,Bexley,Supermarket,Historic Site,Coffee Shop,Train Station,Platform,Bus Stop,Golf Course,Construction & Landscaping,Park,Optical Shop
3,"Bexley, Greenwich",Bus Stop,Construction & Landscaping,Golf Course,Park,Massage Studio,Historic Site,Home Service,Sports Club,Platform,Paper / Office Supplies Store
4,"Bexley, Greenwich",Supermarket,Historic Site,Train Station,Coffee Shop,Platform,Accessories Store,Paper / Office Supplies Store,Pakistani Restaurant,Outdoor Sculpture,Organic Grocery


<h2> Model Build </h2>

We use kmeans clustering to cluster the data

In [121]:
# set number of clusters
k_num_clusters = 5

London_grouped_clustering = London_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_london = KMeans(n_clusters=k_num_clusters, random_state=0).fit(London_grouped_clustering)
kmeans_london

KMeans(n_clusters=5, random_state=0)

In [132]:
kmeans_london.labels_

array([1, 2, 3, 0, 3, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1], dtype=int32)

In [122]:
neighborhoods_venues_sorted_london.insert(0, 'Cluster Labels', kmeans_london.labels_ +1)

Merging the two dataframes

In [123]:
london_data = df1

london_data = london_data.join(neighborhoods_venues_sorted_london.set_index('Neighbourhood'), on='borough')

london_data.head()

,borough,town,post_code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bexley, Greenwich",LONDON,SE2,51.49245,0.12127,4,Supermarket,Historic Site,Train Station,Coffee Shop,Platform,Accessories Store,Paper / Office Supplies Store,Pakistani Restaurant,Outdoor Sculpture,Organic Grocery
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.51324,-0.26746,5,Grocery Store,Park,Indian Restaurant,Breakfast Spot,Train Station,Opera House,Paper / Office Supplies Store,Pakistani Restaurant,Outdoor Sculpture,Organic Grocery
2,City,LONDON,EC3,51.51200,-0.08058,2,Coffee Shop,Hotel,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Restaurant,French Restaurant,Wine Bar,Garden
3,Westminster,LONDON,WC2,51.51651,-0.11968,2,Coffee Shop,Hotel,Pub,Café,Italian Restaurant,Sandwich Place,Hotel Bar,Clothing Store,Theater,Restaurant
4,Bromley,LONDON,SE20,51.41009,-0.05683,1,Supermarket,Convenience Store,Hotel,Fast Food Restaurant,Grocery Store,Park,Italian Restaurant,Pub,Sandwich Place,Historic Site


In [124]:
london_data_nonan = london_data.dropna(subset=['Cluster Labels'])

Let's visualise the clustered neighbourhood

In [166]:
map_clusters_london = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_data_nonan['Latitude'], london_data_nonan['Longitude'], london_data_nonan['borough'], london_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster)) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_london)
        
map_clusters_london

<h3> Let's examine the Clusters </h3>

Cluster 1

In [127]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 1, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,LONDON,1,Supermarket,Convenience Store,Hotel,Fast Food Restaurant,Grocery Store,Park,Italian Restaurant,Pub,Sandwich Place,Historic Site
17,"BECKENHAM, LONDON",1,Supermarket,Convenience Store,Hotel,Fast Food Restaurant,Grocery Store,Park,Italian Restaurant,Pub,Sandwich Place,Historic Site
35,LONDON,1,Fast Food Restaurant,Discount Store,Pharmacy,Chinese Restaurant,Sandwich Place,Indian Restaurant,Bus Stop,Convenience Store,Warehouse Store,Pub
40,LONDON,1,Fast Food Restaurant,Discount Store,Pharmacy,Chinese Restaurant,Sandwich Place,Indian Restaurant,Bus Stop,Convenience Store,Warehouse Store,Pub
61,LONDON,1,Fast Food Restaurant,Discount Store,Pharmacy,Chinese Restaurant,Sandwich Place,Indian Restaurant,Bus Stop,Convenience Store,Warehouse Store,Pub
73,LONDON,1,Supermarket,Convenience Store,Hotel,Fast Food Restaurant,Grocery Store,Park,Italian Restaurant,Pub,Sandwich Place,Historic Site
80,LONDON,1,Fast Food Restaurant,Discount Store,Pharmacy,Chinese Restaurant,Sandwich Place,Indian Restaurant,Bus Stop,Convenience Store,Warehouse Store,Pub
92,"LONDON, WELLING",1,Bus Stop,Construction & Landscaping,Golf Course,Park,Massage Studio,Historic Site,Home Service,Sports Club,Platform,Paper / Office Supplies Store
123,LONDON,1,Fast Food Restaurant,Discount Store,Pharmacy,Chinese Restaurant,Sandwich Place,Indian Restaurant,Bus Stop,Convenience Store,Warehouse Store,Pub
143,LONDON,1,Fast Food Restaurant,Discount Store,Pharmacy,Chinese Restaurant,Sandwich Place,Indian Restaurant,Bus Stop,Convenience Store,Warehouse Store,Pub


Cluster 2

In [128]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 2, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,LONDON,2,Coffee Shop,Hotel,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Restaurant,French Restaurant,Wine Bar,Garden
3,LONDON,2,Coffee Shop,Hotel,Pub,Café,Italian Restaurant,Sandwich Place,Hotel Bar,Clothing Store,Theater,Restaurant
5,LONDON,2,Coffee Shop,Pub,Café,Food Truck,Vietnamese Restaurant,Park,Italian Restaurant,Hotel,Gym / Fitness Center,Cocktail Bar
6,LONDON,2,Coffee Shop,Pub,Café,Food Truck,Vietnamese Restaurant,Park,Italian Restaurant,Hotel,Gym / Fitness Center,Cocktail Bar
7,"BARNET, LONDON",2,Coffee Shop,Café,Grocery Store,Pub,Italian Restaurant,Bus Stop,Supermarket,Pharmacy,Turkish Restaurant,Chinese Restaurant
8,LONDON,2,Italian Restaurant,Coffee Shop,Pizza Place,Café,Fast Food Restaurant,Supermarket,Pub,Grocery Store,Turkish Restaurant,Gym / Fitness Center
9,LONDON,2,Pub,Coffee Shop,Indian Restaurant,Supermarket,Café,Portuguese Restaurant,Grocery Store,Bar,Burger Joint,Gym
10,LONDON,2,Pub,Café,Coffee Shop,Grocery Store,Park,Gym / Fitness Center,Pharmacy,Bar,Italian Restaurant,Vietnamese Restaurant
11,LONDON,2,Coffee Shop,Hotel,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Restaurant,French Restaurant,Wine Bar,Garden
12,LONDON,2,Coffee Shop,Pub,Pizza Place,Café,Grocery Store,Park,Farmers Market,Italian Restaurant,Middle Eastern Restaurant,Track


Cluster 3 

In [129]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 3, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,LONDON,3,Gym / Fitness Center,Hardware Store,Clothing Store,Supermarket,Opera House,Park,Paper / Office Supplies Store,Pakistani Restaurant,Outdoor Sculpture,Organic Grocery


Cluster 4

In [130]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 4, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,LONDON,4,Supermarket,Historic Site,Train Station,Coffee Shop,Platform,Accessories Store,Paper / Office Supplies Store,Pakistani Restaurant,Outdoor Sculpture,Organic Grocery
25,"BEXLEYHEATH, LONDON",4,Supermarket,Historic Site,Coffee Shop,Train Station,Platform,Bus Stop,Golf Course,Construction & Landscaping,Park,Optical Shop
71,LONDON,4,Supermarket,Historic Site,Coffee Shop,Train Station,Platform,Bus Stop,Golf Course,Construction & Landscaping,Park,Optical Shop
165,"LONDON, SIDCUP",4,Supermarket,Historic Site,Coffee Shop,Train Station,Platform,Bus Stop,Golf Course,Construction & Landscaping,Park,Optical Shop
290,LONDON,4,Supermarket,Historic Site,Coffee Shop,Train Station,Platform,Bus Stop,Golf Course,Construction & Landscaping,Park,Optical Shop


Cluster 5

In [131]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 5, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,LONDON,5,Grocery Store,Park,Indian Restaurant,Breakfast Spot,Train Station,Opera House,Paper / Office Supplies Store,Pakistani Restaurant,Outdoor Sculpture,Organic Grocery


<h3> Exploring Paris </h3>

In [133]:
!wget -q -O 'france-data.json' https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e
print("Data Downloaded!")

Data Downloaded!


In [134]:
paris_raw = pd.read_json('france-data.json')
paris_raw.head()

,datasetid,recordid,fields,geometry,record_timestamp
0,correspondances-code-insee-code-postal,2bf36b38314b6c39dfbcd09225f97fa532b1fc45,"{'code_comm': '645', 'nom_dept': 'ESSONNE', 'statut': 'Commune simple', 'z_moyen': 121.0, 'nom_region': 'ILE-DE-FRANCE', 'code_reg': '11', 'insee_com': '91645', 'code_dept': '91', 'geo_point_2d': [48.750443119964764, 2.251712972144151], 'postal_code': '91370', 'id_geofla': '16275', 'code_cant': '03', 'geo_shape': {'type': 'Polygon', 'coordinates': [[[2.238024349288764, 48.735565859837095], [2.226414985434264, 48.75003536744732], [2.22450256558849, 48.75882853410981], [2.232859032169924, 48.76598806763034], [2.250043759055985, 48.761213267519565], [2.269288614654887, 48.76063999654954], [2....","{'type': 'Point', 'coordinates': [2.251712972144151, 48.750443119964764]}",2016-09-21T00:29:06.175+02:00
1,correspondances-code-insee-code-postal,7ee82e74e059b443df18bb79fc5a19b1f05e5a88,"{'code_comm': '133', 'nom_dept': 'SEINE-ET-MARNE', 'statut': 'Commune simple', 'z_moyen': 88.0, 'nom_region': 'ILE-DE-FRANCE', 'code_reg': '11', 'insee_com': '77133', 'code_dept': '77', 'geo_point_2d': [48.41256065214989, 3.052940505560729], 'postal_code': '77126', 'id_geofla': '31428', 'code_cant': '20', 'geo_shape': {'type': 'Polygon', 'coordinates': [[[3.076046701822989, 48.397361878531605], [3.063543312872447, 48.40290215525443], [3.044758381476711, 48.3978276484716], [3.034761984122421, 48.39926146328478], [3.020642255927941, 48.39551316759141], [3.010956509025493, 48.39763766392435],...","{'type': 'Point', 'coordinates': [3.052940505560729, 48.41256065214989]}",2016-09-21T00:29:06.175+02:00
2,correspondances-code-insee-code-postal,e2cd3186f07286705ed482a10b6aebd9de633c81,"{'code_comm': '378', 'nom_dept': 'ESSONNE', 'statut': 'Commune simple', 'z_moyen': 150.0, 'nom_region': 'ILE-DE-FRANCE', 'code_reg': '11', 'insee_com': '91378', 'code_dept': '91', 'geo_point_2d': [48.52726809075556, 2.19718165044305], 'postal_code': '91730', 'id_geofla': '30975', 'code_cant': '09', 'geo_shape': {'type': 'Polygon', 'coordinates': [[[2.203466690733517, 48.51655284725087], [2.194862275092367, 48.51516342092208], [2.187298703016565, 48.518287625065426], [2.189277035188291, 48.534854146313315], [2.195177412526977, 48.53963518797898], [2.192640118891314, 48.5425327211163], [2.20...","{'type': 'Point', 'coordinates': [2.19718165044305, 48.52726809075556]}",2016-09-21T00:29:06.175+02:00
3,correspondances-code-insee-code-postal,868bf03527a1d0a9defe5cf4e6fa0a730d725699,"{'code_comm': '243', 'nom_dept': 'SEINE-ET-MARNE', 'statut': 'Chef-lieu canton', 'z_moyen': 71.0, 'nom_region': 'ILE-DE-FRANCE', 'code_reg': '11', 'insee_com': '77243', 'code_dept': '77', 'geo_point_2d': [48.87307018579678, 2.7097808131278462], 'postal_code': '77400', 'id_geofla': '17000', 'code_cant': '14', 'geo_shape': {'type': 'Polygon', 'coordinates': [[[2.727542158243183, 48.85975862454365], [2.709454819890855, 48.864761276749235], [2.696453631985896, 48.866608587950175], [2.694166759216122, 48.8730238890733], [2.680162376432985, 48.87129657626423], [2.673953998830155, 48.876522830016...","{'type': 'Point', 'coordinates': [2.7097808131278462, 48.87307018579678]}",2016-09-21T00:29:06.175+02:00
4,correspondances-code-insee-code-postal,21e809b1d4480333c8b6fe7addd8f3b06f343e2c,"{'code_comm': '003', 'nom_dept': 'VAL-DE-MARNE', 'statut': 'Chef-lieu canton', 'z_moyen': 70.0, 'nom_region': 'ILE-DE-FRANCE', 'code_reg': '11', 'insee_com': '94003', 'code_dept': '94', 'geo_point_2d': [48.80588035965699, 2.333510249842654], 'postal_code': '94110', 'id_geofla': '32123', 'code_cant': '34', 'geo_shape': {'type': 'Polygon', 'coordinates': [[[2.34385114554979, 48.79766105911435], [2.323937975769717, 48.801552407915224], [2.31904946488543, 48.80995199111788], [2.329063222575344, 48.81378604475913], [2.337884503474758, 48.812971810560256], [2.344856549104409, 48.80541037414988],...","{'type': 'Point', 'coordinates': [2.333510249842654, 48.80588035965699]}",2016-09-21T00:29:06.175+02:00


In [135]:
paris_field_data = pd.DataFrame()
for f in paris_raw.fields:
    dict_new = f
    paris_field_data = paris_field_data.append(dict_new, ignore_index=True)

paris_field_data.head()

,code_arr,code_cant,code_comm,code_dept,code_reg,geo_point_2d,geo_shape,id_geofla,insee_com,nom_comm,nom_dept,nom_region,population,postal_code,statut,superficie,z_moyen
0,3,03,645,91,11,"[48.750443119964764, 2.251712972144151]","{'type': 'Polygon', 'coordinates': [[[2.238024349288764, 48.735565859837095], [2.226414985434264, 48.75003536744732], [2.22450256558849, 48.75882853410981], [2.232859032169924, 48.76598806763034], [2.250043759055985, 48.761213267519565], [2.269288614654887, 48.76063999654954], [2.276145972515501, 48.75666127305422], [2.283691112862691, 48.748081131389654], [2.274517407535147, 48.74072222671912], [2.238024349288764, 48.735565859837095]]]}",16275,91645,VERRIERES-LE-BUISSON,ESSONNE,ILE-DE-FRANCE,15.5,91370,Commune simple,999.0,121.0
1,3,20,133,77,11,"[48.41256065214989, 3.052940505560729]","{'type': 'Polygon', 'coordinates': [[[3.076046701822989, 48.397361878531605], [3.063543312872447, 48.40290215525443], [3.044758381476711, 48.3978276484716], [3.034761984122421, 48.39926146328478], [3.020642255927941, 48.39551316759141], [3.010956509025493, 48.39763766392435], [3.027765439177065, 48.402646031453706], [3.027972845131529, 48.411552580260555], [3.040895342404548, 48.416227017670224], [3.04385481424453, 48.432140675154415], [3.061525874606292, 48.43389612009283], [3.068145553754999, 48.43024896325371], [3.068528666803148, 48.42341145518176], [3.083214183756346, 48.4019633336603...",31428,77133,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,ILE-DE-FRANCE,0.2,77126,Commune simple,1082.0,88.0
2,1,09,378,91,11,"[48.52726809075556, 2.19718165044305]","{'type': 'Polygon', 'coordinates': [[[2.203466690733517, 48.51655284725087], [2.194862275092367, 48.51516342092208], [2.187298703016565, 48.518287625065426], [2.189277035188291, 48.534854146313315], [2.195177412526977, 48.53963518797898], [2.192640118891314, 48.5425327211163], [2.208230369748835, 48.533785391795064], [2.203466690733517, 48.51655284725087]]]}",30975,91378,MAUCHAMPS,ESSONNE,ILE-DE-FRANCE,0.3,91730,Commune simple,313.0,150.0
3,5,14,243,77,11,"[48.87307018579678, 2.7097808131278462]","{'type': 'Polygon', 'coordinates': [[[2.727542158243183, 48.85975862454365], [2.709454819890855, 48.864761276749235], [2.696453631985896, 48.866608587950175], [2.694166759216122, 48.8730238890733], [2.680162376432985, 48.87129657626423], [2.673953998830155, 48.87652283001623], [2.703843665050603, 48.88069236307745], [2.727733026974796, 48.883222165132516], [2.73488085797555, 48.882707616585115], [2.729823867621858, 48.882345582776395], [2.729660828405411, 48.869098369135386], [2.727542158243183, 48.85975862454365]]]}",17000,77243,LAGNY-SUR-MARNE,SEINE-ET-MARNE,ILE-DE-FRANCE,20.2,77400,Chef-lieu canton,579.0,71.0
4,3,34,003,94,11,"[48.80588035965699, 2.333510249842654]","{'type': 'Polygon', 'coordinates': [[[2.34385114554979, 48.79766105911435], [2.323937975769717, 48.801552407915224], [2.31904946488543, 48.80995199111788], [2.329063222575344, 48.81378604475913], [2.337884503474758, 48.812971810560256], [2.344856549104409, 48.80541037414988], [2.347778462376922, 48.80149599950683], [2.34385114554979, 48.79766105911435]]]}",32123,94003,ARCUEIL,VAL-DE-MARNE,ILE-DE-FRANCE,19.5,94110,Chef-lieu canton,232.0,70.0


We only take the columns we need

In [137]:
df2 = paris_field_data[['postal_code','nom_comm','nom_dept','geo_point_2d']]
df2.head()

,postal_code,nom_comm,nom_dept,geo_point_2d
0,91370,VERRIERES-LE-BUISSON,ESSONNE,"[48.750443119964764, 2.251712972144151]"
1,77126,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,"[48.41256065214989, 3.052940505560729]"
2,91730,MAUCHAMPS,ESSONNE,"[48.52726809075556, 2.19718165044305]"
3,77400,LAGNY-SUR-MARNE,SEINE-ET-MARNE,"[48.87307018579678, 2.7097808131278462]"
4,94110,ARCUEIL,VAL-DE-MARNE,"[48.80588035965699, 2.333510249842654]"


We only want the Paris data...

In [138]:
df_paris = df2[df2['nom_dept'].str.contains('PARIS')].reset_index(drop=True)
df_paris.head()

,postal_code,nom_comm,nom_dept,geo_point_2d
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,"[48.87689616237872, 2.337460241388529]"
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,"[48.86790337886785, 2.344107166658533]"
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,"[48.85941549762748, 2.378741060237548]"
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,"[48.86305413181178, 2.359361058970589]"
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,"[48.84896809191946, 2.332670898588416]"


In [139]:
df_paris.shape

(20, 4)

In [140]:
df_paris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   postal_code   20 non-null     object
 1   nom_comm      20 non-null     object
 2   nom_dept      20 non-null     object
 3   geo_point_2d  20 non-null     object
dtypes: object(4)
memory usage: 768.0+ bytes


Now to get the coordinates

In [141]:
paris_latlng = df_paris['geo_point_2d'].astype('str')

In [142]:
#latitude
paris_lat = paris_latlng.apply(lambda x: x.split(',')[0])
paris_lat = paris_lat.apply(lambda x: x.lstrip('['))

#longitude
paris_lng = paris_latlng.apply(lambda x: x.split(',')[1])
paris_lng = paris_lng.apply(lambda x: x.rstrip(']'))

print(paris_lat,paris_lng)

0      48.87689616237872
1      48.86790337886785
2      48.85941549762748
3      48.86305413181178
4      48.84896809191946
5     48.854228281954754
6      48.87602855694339
7      48.86039876035177
8      48.87252726662346
9      48.82871768452136
10     48.83515623066034
11    48.844508659617546
12     48.88686862295828
13     48.86318677744551
14     48.85608259819694
15    48.892735074561706
16     48.88733716648682
17     48.84015541860987
18      48.8626304851685
19     48.82899321160942
Name: geo_point_2d, dtype: object 0      2.337460241388529
1      2.344107166658533
2      2.378741060237548
3      2.359361058970589
4      2.332670898588416
5      2.357361938142205
6      2.361112904561707
7      2.262099559395783
8      2.312582560420059
9      2.362468228516128
10     2.419807034965275
11     2.349859385560182
12     2.384694327870042
13     2.400819826729021
14     2.312438687733857
15     2.348711933867703
16     2.307485559493426
17     2.293559372435076
18     2.3362934

In [143]:
paris_geo_lat  = pd.DataFrame(paris_lat.astype(float))
paris_geo_lat.columns=['Latitude']
paris_geo_lng = pd.DataFrame(paris_lng.astype(float))
paris_geo_lng.columns=['Longitude']

paris_combined_data = pd.concat([df_paris.drop('geo_point_2d', axis=1), paris_geo_lat, paris_geo_lng], axis=1)
paris_combined_data.head()

,postal_code,nom_comm,nom_dept,Latitude,Longitude
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671


In [144]:
paris_combined_data.isna().sum()

postal_code    0
nom_comm       0
nom_dept       0
Latitude       0
Longitude      0
dtype: int64

Visualising Paris

In [145]:
paris_lat_coords = '48.85341000000005'
paris_lng_coords = '2.3488000000000397'


# Creating the map of Paris
map_Paris= folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)
map_Paris

# adding markers to map
for latitude, longitude, borough, town in zip(paris_combined_data['Latitude'], paris_combined_data['Longitude'], paris_combined_data['nom_comm'], paris_combined_data['nom_dept']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='Blue',
        fill=True,
        fill_opacity=0.8
        ).add_to(map_Paris)  
    
map_Paris

Let us fetch the venues in Paris

In [146]:
venues_in_Paris = getNearbyVenues(paris_combined_data['nom_comm'], paris_combined_data['Latitude'], paris_combined_data['Longitude'])

PARIS-9E-ARRONDISSEMENT
PARIS-2E-ARRONDISSEMENT
PARIS-11E-ARRONDISSEMENT
PARIS-3E-ARRONDISSEMENT
PARIS-6E-ARRONDISSEMENT
PARIS-4E-ARRONDISSEMENT
PARIS-10E-ARRONDISSEMENT
PARIS-16E-ARRONDISSEMENT
PARIS-8E-ARRONDISSEMENT
PARIS-13E-ARRONDISSEMENT
PARIS-12E-ARRONDISSEMENT
PARIS-5E-ARRONDISSEMENT
PARIS-19E-ARRONDISSEMENT
PARIS-20E-ARRONDISSEMENT
PARIS-7E-ARRONDISSEMENT
PARIS-18E-ARRONDISSEMENT
PARIS-17E-ARRONDISSEMENT
PARIS-15E-ARRONDISSEMENT
PARIS-1ER-ARRONDISSEMENT
PARIS-14E-ARRONDISSEMENT


In [147]:
venues_in_Paris.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Place Saint-Georges,Plaza
1,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Farine & O,Bakery
2,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,RAP,Gourmet Shop
3,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Le Bouclier de Bacchus,Wine Bar
4,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,So Nat,Vegetarian / Vegan Restaurant


In [148]:
venues_in_Paris.shape

(1276, 5)

One-hot encoding the venue categories for clustering

In [150]:
Paris_venue_cat = pd.get_dummies(venues_in_Paris[['Venue Category']], prefix="", prefix_sep="")
Paris_venue_cat.head()

,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auvergne Restaurant,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Café,Cambodian Restaurant,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,EV Charging Station,Electronics Store,Escape Room,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,New American Restaurant,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Snack Place,Soba Restaurant,South American Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [151]:
Paris_venue_cat['Neighbourhood'] = venues_in_Paris['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [Paris_venue_cat.columns[-1]] + list(Paris_venue_cat.columns[:-1])
Paris_venue_cat = Paris_venue_cat[fixed_columns]

Paris_venue_cat.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auvergne Restaurant,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Café,Cambodian Restaurant,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,EV Charging Station,Electronics Store,Escape Room,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,New American Restaurant,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Snack Place,Soba Restaurant,South American Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

get mean values for venue categories in each neighbourhood

In [152]:
Paris_grouped = Paris_venue_cat.groupby('Neighbourhood').mean().reset_index()
Paris_grouped.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auvergne Restaurant,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Café,Cambodian Restaurant,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,EV Charging Station,Electronics Store,Escape Room,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,New American Restaurant,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Snack Place,Soba Restaurant,South American Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,PARIS-10E-ARRONDISSEMENT,0.000,0.02,0.0,0.0,0.0,0.0,0.000,0.0,0.030000,0.0,0.0,0.0,0.020000,0.020,0.0,0.0,0.0,0.01,0.0,0.0,0.060000,0.0,0.010000,0.0,0.01,0.000000,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.040000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.010,0.050000,0.0,0.000,0.0,0.000000,0.0,0.01,0.010000,0.0,0.0,0.0,0.0,0.000000,0.0,0.01,0.01,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.01,0.010000,0.0,0.000,0.01,0.010000,0.0,0.110000,0.0,0.010000,0.01,0.0,0.000,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.01,0.050000,0.0,0.01,0.02,0.0,0.0,0.01,0.030000,0.030000,0.0,0.0,0.010000,0.020,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.02,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.030000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.01,0.0,0.010000,0.0,0.0,0.02,0.01,0.0,0.01,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.000000,0.01,0.0,0.01,0.0,0.0,0.00000

Getting top venue categories

In [153]:
# create a new dataframe for Paris
neighborhoods_venues_sorted_paris = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_paris['Neighbourhood'] = Paris_grouped['Neighbourhood']

for ind in np.arange(Paris_grouped.shape[0]):
    neighborhoods_venues_sorted_paris.iloc[ind, 1:] = return_most_common_venues(Paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_paris.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-10E-ARRONDISSEMENT,French Restaurant,Bistro,Coffee Shop,Hotel,Café,Italian Restaurant,Pizza Place,Asian Restaurant,Japanese Restaurant,Bar
1,PARIS-11E-ARRONDISSEMENT,Restaurant,Café,Italian Restaurant,Bistro,French Restaurant,Pastry Shop,Bakery,Vietnamese Restaurant,Sandwich Place,Plaza
2,PARIS-12E-ARRONDISSEMENT,Zoo Exhibit,Bistro,Monument / Landmark,Supermarket,Zoo,Juice Bar,Music Store,Noodle House,Okonomiyaki Restaurant,Optical Shop
3,PARIS-13E-ARRONDISSEMENT,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,French Restaurant,Juice Bar,Italian Restaurant,Creperie,Gourmet Shop,Furniture / Home Store
4,PARIS-14E-ARRONDISSEMENT,French Restaurant,Hotel,Bakery,Italian Restaurant,EV Charging Station,Plaza,Pizza Place,Fast Food Restaurant,Food & Drink Shop,Café


<h3> Model Build </h3>

In [154]:
# set number of clusters
k_num_clusters = 5

Paris_grouped_clustering = Paris_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_Paris = KMeans(n_clusters=k_num_clusters, random_state=0).fit(Paris_grouped_clustering)
kmeans_Paris

KMeans(n_clusters=5, random_state=0)

In [155]:
kmeans_Paris.labels_

array([0, 0, 2, 3, 4, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0],
      dtype=int32)

In [156]:
neighborhoods_venues_sorted_paris.insert(0, 'Cluster Labels', kmeans_Paris.labels_ +1)

In [157]:
paris_data = paris_combined_data

paris_data = paris_data.join(neighborhoods_venues_sorted_paris.set_index('Neighbourhood'), on='nom_comm')

paris_data.head()

,postal_code,nom_comm,nom_dept,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460,1,French Restaurant,Hotel,Bistro,Japanese Restaurant,Wine Bar,Bakery,Cocktail Bar,Lounge,Restaurant,Plaza
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107,1,French Restaurant,Cocktail Bar,Wine Bar,Bakery,Hotel,Coffee Shop,Sandwich Place,Clothing Store,Pastry Shop,Indie Movie Theater
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741,1,Restaurant,Café,Italian Restaurant,Bistro,French Restaurant,Pastry Shop,Bakery,Vietnamese Restaurant,Sandwich Place,Plaza
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361,1,French Restaurant,Coffee Shop,Art Gallery,Bakery,Italian Restaurant,Cocktail Bar,Gourmet Shop,Bookstore,Wine Bar,Burger Joint
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671,1,Bakery,Chocolate Shop,French Restaurant,Cocktail Bar,Wine Bar,Restaurant,Pastry Shop,Fountain,Pharmacy,Clothing Store


In [158]:
paris_data_nonan = paris_data.dropna(subset=['Cluster Labels'])

Now visualise the clustered neighbourhoods

In [162]:
map_clusters_paris = folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_data_nonan['Latitude'], paris_data_nonan['Longitude'], paris_data_nonan['nom_comm'], paris_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) ) + ' ' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.8
        ).add_to(map_clusters_paris)
        
map_clusters_paris

Let us examine the clusters

Cluster 1

In [160]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 1, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-9E-ARRONDISSEMENT,1,French Restaurant,Hotel,Bistro,Japanese Restaurant,Wine Bar,Bakery,Cocktail Bar,Lounge,Restaurant,Plaza
1,PARIS-2E-ARRONDISSEMENT,1,French Restaurant,Cocktail Bar,Wine Bar,Bakery,Hotel,Coffee Shop,Sandwich Place,Clothing Store,Pastry Shop,Indie Movie Theater
2,PARIS-11E-ARRONDISSEMENT,1,Restaurant,Café,Italian Restaurant,Bistro,French Restaurant,Pastry Shop,Bakery,Vietnamese Restaurant,Sandwich Place,Plaza
3,PARIS-3E-ARRONDISSEMENT,1,French Restaurant,Coffee Shop,Art Gallery,Bakery,Italian Restaurant,Cocktail Bar,Gourmet Shop,Bookstore,Wine Bar,Burger Joint
4,PARIS-6E-ARRONDISSEMENT,1,Bakery,Chocolate Shop,French Restaurant,Cocktail Bar,Wine Bar,Restaurant,Pastry Shop,Fountain,Pharmacy,Clothing Store
5,PARIS-4E-ARRONDISSEMENT,1,French Restaurant,Ice Cream Shop,Clothing Store,Hotel,Park,Pastry Shop,Pedestrian Plaza,Wine Bar,Coffee Shop,Tea Room
6,PARIS-10E-ARRONDISSEMENT,1,French Restaurant,Bistro,Coffee Shop,Hotel,Café,Italian Restaurant,Pizza Place,Asian Restaurant,Japanese Restaurant,Bar
11,PARIS-5E-ARRONDISSEMENT,1,French Restaurant,Hotel,Italian Restaurant,Plaza,Bakery,Café,Coffee Shop,Science Museum,Pub,Vietnamese Restaurant
12,PARIS-19E-ARRONDISSEMENT,1,French Restaurant,Bar,Hotel,Brewery,Bistro,Seafood Restaurant,Beer Bar,Supermarket,Café,Harbor / Marina
13,PARIS-20E-ARRONDISSEMENT,1,Bakery,Plaza,French Restaurant,Japanese Restaurant,Italian Restaurant,Bistro,Bar,Café,Pizza Place,Hotel


Cluster 2

In [161]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 2, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,PARIS-16E-ARRONDISSEMENT,2,Lake,Plaza,French Restaurant,Art Museum,Bus Station,Boat or Ferry,Park,Afghan Restaurant,Performing Arts Venue,Okonomiyaki Restaurant


Cluster 3

In [163]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 3, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,PARIS-12E-ARRONDISSEMENT,3,Zoo Exhibit,Bistro,Monument / Landmark,Supermarket,Zoo,Juice Bar,Music Store,Noodle House,Okonomiyaki Restaurant,Optical Shop


Cluster 4

In [164]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 4, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,PARIS-13E-ARRONDISSEMENT,4,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,French Restaurant,Juice Bar,Italian Restaurant,Creperie,Gourmet Shop,Furniture / Home Store


Cluster 5

In [165]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 5, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,PARIS-8E-ARRONDISSEMENT,5,French Restaurant,Hotel,Bakery,Corsican Restaurant,Spa,Art Gallery,Hotel Bar,Park,Sporting Goods Shop,Furniture / Home Store
14,PARIS-7E-ARRONDISSEMENT,5,French Restaurant,Hotel,Café,Italian Restaurant,Plaza,Cocktail Bar,History Museum,Cheese Shop,Park,Historic Site
16,PARIS-17E-ARRONDISSEMENT,5,French Restaurant,Hotel,Italian Restaurant,Café,Bakery,Bistro,Plaza,Japanese Restaurant,Restaurant,Pizza Place
19,PARIS-14E-ARRONDISSEMENT,5,French Restaurant,Hotel,Bakery,Italian Restaurant,EV Charging Station,Plaza,Pizza Place,Fast Food Restaurant,Food & Drink Shop,Café


<h1> Conclusion </h1>

<p> he purpose of this project was to explore the cities of London and Paris and see how attractive it is to potential tourists and migrants. We explored both the cities based on their postal codes and then extrapolated the common venues present in each of the neighbourhoods finally concluding with clustering similar neighbourhoods together. </p>

<p> We could see that each of the neighbourhoods in both the cities have a wide variety of experiences to offer which is unique in it's own way.It's upto the stakeholders to decide which experience they would prefer more. </p>